In [0]:
import torch
import matplotlib.pyplot as plt
import numpy as np

from torch import nn
from torch.nn import functional as F
from torchvision import datasets, transforms

import PIL
from PIL import Image
from torch.utils import data as D
from torch.utils.data.sampler import SubsetRandomSampler
import random

transformer = transforms.Compose([transforms.Resize((32, 32)),
                                  transforms.ToTensor(),
                                  transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                                 ])

In [77]:
trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transformer)

validset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transformer)

testset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transformer)

num_train = len(trainset)
indices = list(range(num_train))
split = int(np.floor(0.1 * num_train))

np.random.seed(10)
np.random.shuffle(indices)

train_idx, valid_idx = indices[split:], indices[:split]
train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

train_loader = torch.utils.data.DataLoader(dataset=trainset, batch_size=100, sampler=train_sampler)

valid_loader = torch.utils.data.DataLoader(dataset=validset, batch_size=100, sampler=valid_sampler)

test_loader = torch.utils.data.DataLoader(dataset=testset, batch_size=100, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [0]:
class LeNet(nn.Module):
  
  def __init__(self):
    super().__init__()
   
    self.fc1 = nn.Linear(3072, 768)
    self.fc2 = nn.Linear(768, 384)
    self.fc3=nn.Linear(384,10)

  def forward(self, x):
   
    x = x.view(-1, 3072)
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    return x

  
model = LeNet()
device=torch.device(
    "cuda:0" if torch.cuda.is_available() else "cpu")

In [79]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

epochs = 10
running_loss_history = []
running_correct_history = []
validation_running_loss_history = [] 
validation_running_correct_history = []

for e in range(epochs):  
    
    running_loss = 0.0
    running_correct = 0.0
    validation_running_loss = 0.0
    validation_running_correct = 0.0

    # training part  
    for inputs, labels in train_loader:

        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = model(inputs)
        loss = criterion(outputs, labels)
    
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
        _, preds = torch.max(outputs, 1)   
        running_correct += torch.sum(preds == labels.data)
        running_loss += loss.item()

        epoch_loss = running_loss / len(train_loader)
        epoch_acc = running_correct.float() / len(train_loader)
        running_loss_history.append(epoch_loss)
        running_correct_history.append(epoch_acc)

              
  # validation part
   
    for i, data in valid_loader:
        val_inputs = i.to(device)
        val_labels = data.to(device)
        val_outputs = model(val_inputs)
        val_loss = criterion(outputs, labels)

        _, val_preds = torch.max(val_outputs, 1)
        validation_running_loss += val_loss.item()
        validation_running_correct += torch.sum(val_preds == val_labels.data)

        val_epoch_loss = validation_running_loss / len(valid_loader)
        val_epoch_acc = validation_running_correct.float() / len(valid_loader)
        validation_running_loss_history.append(val_epoch_loss)
        validation_running_correct_history.append(val_epoch_acc) 

    print("epoch: ", e + 1)
    print("training loss: {:.5f}, acc: {:5f}".format(epoch_loss, epoch_acc))
    print("validation loss: {:.5f}, acc: {:5f}".format(val_epoch_loss, val_epoch_acc))
        

epoch:  1
training loss: 1.64791, acc: 41.459999
validation loss: 1.44578, acc: 46.320000
epoch:  2
training loss: 1.43183, acc: 49.255554
validation loss: 1.23155, acc: 49.480000
epoch:  3
training loss: 1.31455, acc: 53.515556
validation loss: 1.38003, acc: 52.180000
epoch:  4
training loss: 1.21027, acc: 57.168888
validation loss: 1.16927, acc: 51.720001
epoch:  5
training loss: 1.12245, acc: 60.046665
validation loss: 1.24256, acc: 53.160000
epoch:  6
training loss: 1.03499, acc: 63.062222
validation loss: 1.06653, acc: 52.459999
epoch:  7
training loss: 0.94680, acc: 66.473335
validation loss: 0.87105, acc: 52.419998
epoch:  8
training loss: 0.86172, acc: 69.428886
validation loss: 0.66146, acc: 53.040001
epoch:  9
training loss: 0.78718, acc: 71.986664
validation loss: 0.84031, acc: 54.720001
epoch:  10
training loss: 0.70198, acc: 75.033333
validation loss: 0.81719, acc: 53.560001


In [80]:
correct = 0
total = 0

# test part
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
                
        for i in range(labels.shape[0]):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1
            
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

print('Accuracy of the test images: %d %%' % (100 * correct / total))            

Accuracy of the test images: 53 %
